In [1]:
import spacy
import pickle
activated = spacy.prefer_gpu()
nlp = spacy.load('en_core_web_lg')

In [5]:
# ex = nlp("Understand customer needs and develop high-quality presentations, proposals and software demonstrations that speak to these needs - building a consensus for change on a multi-stakeholder basis")
# ex = nlp("Need to understand and demonstrate the solution and how to integrate to other business systems")

for chunk in ex.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

the solution solution dobj demonstrate
other business systems systems pobj to


In [17]:
# Split VP phrases from the sentence
def split_VP_phrases(doc):
    print("Base ===== ", doc.text)
    res = []
    root = doc[:].root
    
    def check_conj(rt):
        for token in rt.rights:
            if token.dep_ not in ['cc', 'conj', 'punct']:
                return False
            if token.dep_ == 'conj':
                break
        return True
    
    def split(rt):   
        if rt.pos_ == 'PUNCT' or rt.pos_ == 'CCONJ' or rt.pos_ == 'SYM':
            return []
        ret = ['']
        if not (rt.pos_ == 'VERB' and rt.dep_ not in ['xcomp', 'ccomp']):
            for token in rt.lefts:
                left_ret = split(token)
                if len(left_ret) == 0:
                    continue
                ret_ = [x + y + " " for x in ret for y in left_ret]
                ret = ret_
        
        ret_ = [x + rt.text for x in ret]
        ret = []
     
        flag = False
        for token in rt.rights:
            right_ret = split(token)
           
            if len(right_ret) == 0:
                continue
            
            if token.dep_ == 'conj' and rt.pos_ == token.pos_ and check_conj(rt):
               
                if not flag:
                    for sent in right_ret:
                      
                        right = doc[token.left_edge.i : token.i+1].text
                        if right in sent:
                            ret.extend([sent.replace(right, x) for x in ret_])
                    flag = True
                if token.pos_ != 'VERB':
                    ret.extend(right_ret)
                ret_ = ret
               
            else:
                ret = [x + " " + y for x in ret_ for y in right_ret]
                ret_ = ret
        ret = ret_
      
        if rt.pos_ == 'VERB' and rt.dep_ not in ['xcomp', 'ccomp']:
            res.extend(ret)
            if rt.dep_ == 'conj' and rt.head.pos_ == 'VERB' and check_conj(rt.head):
                return ret
            else:
                return []
        else:
            if rt.i == root.i:
                res.extend(ret)
            return ret
        
    split(root)
    
    # Normalize the sentence
    res_ = []
    for sent in res:
        sent = nlp(sent)
        if len(sent) < 2:
            continue    
        res_.append(sent.text.replace(sent[0].text, sent[0].lemma_).strip())
    print('\n'.join(res_))
    return res_
ex = nlp("Collaborate with other AWS specialist teams to drive and shape solutions to accelerate our customer's business outcomes")    
# ex = nlp("Need to understand and demonstrate the solution and how to integrate to other business systems")
from spacy import displacy
displacy.render(ex, style = "dep", jupyter = True)
ret = split_VP_phrases(ex)

Base =====  Collaborate with other AWS specialist teams to drive and shape solutions to accelerate our customer's business outcomes
drive shape
accelerate our customer 's business outcomes
collaborate with other AWS specialist teams solutions


In [12]:
for token in ex:
    print(token.dep_)

ROOT
cc
conj
prep
pobj
punct
acl
compound
dobj
punct
conj
det
compound
dobj
cc
conj


In [14]:
spacy.explain("ADP")

'adposition'

In [2]:
import textacy

In [3]:
example = "Textacy is a Python library for performing higher-level natural language processing (NLP) tasks, built on the high-performance Spacy library. With the basics — tokenization, part-of-speech tagging, parsing — offloaded to another library, textacy focuses on tasks facilitated by the availability of tokenized, POS-tagged, and parsed text: keyterm extraction, readability statistics, emotional valence analysis, quotation attribution, and more."

In [12]:
raw_text = """Experience in two or more of the following transformation practice areas: ERP (business process architecture and solution implementation), business case and use case development, financial modelling, BI and Analytics strategy and implementation, complex program management, enterprise business / process / IT architectures, executive agenda items and metrics."""

In [13]:
raw_text

'Experience in two or more of the following transformation practice areas: ERP (business process architecture and solution implementation), business case and use case development, financial modelling, BI and Analytics strategy and implementation, complex program management, enterprise business / process / IT architectures, executive agenda items and metrics.'

In [16]:
doc = textacy.make_spacy_doc(raw_text)

In [15]:
from textacy import preprocessing

In [17]:
doc._.preview

'Doc(57 tokens: "Experience in two or more of the following tran...")'

In [19]:
pattern = textacy.constants.POS_REGEX_PATTERNS["en"]["NP"]
pattern

'<DET>? <NUM>* (<ADJ> <PUNCT>? <CONJ>?)* (<NOUN>|<PROPN> <PART>?)+'

In [22]:
ts = textacy.TextStats(doc)

In [23]:
ts.readability_stats

{'flesch_kincaid_grade_level': 27.395555555555564,
 'flesch_reading_ease': -21.19999999999996,
 'smog_index': 25.98008509911553,
 'gunning_fog_index': 32.22222222222222,
 'coleman_liau_index': 22.871370911111114,
 'automated_readability_index': 32.574666666666666,
 'lix': 100.55555555555556,
 'gulpease_index': 28.77777777777778,
 'wiener_sachtextformel': 19.281222222222222}

In [24]:
ts.basic_counts

{'n_sents': 1,
 'n_words': 45,
 'n_chars': 301,
 'n_syllables': 97,
 'n_unique_words': 36,
 'n_long_words': 25,
 'n_monosyllable_words': 20,
 'n_polysyllable_words': 16}

In [4]:
import textacy.resources
rs = textacy.resources.ConceptNet()
rs.download()
rs.info

{'name': 'concept_net',
 'site_url': 'http://conceptnet.io',
 'publication_url': 'https://arxiv.org/abs/1612.03975',
 'description': 'An open, multilingual semantic network of general knowledge, designed to help computers understand the meanings of words.'}

In [10]:
rs.get_antonyms("go", lang="en", sense="v")

757it [00:00, 84333.93it/s]


ValueError: not enough values to unpack (expected 5, got 4)